In [1]:
import sys
sys.path.append('./pytorch_sol2/')
#sys.path

In [2]:
from __future__ import print_function, division
import os, time, copy, sys
import pandas as pd
import numpy as np
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from skimage import io, transform
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import pickle
from pandas_ml import ConfusionMatrix
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from config import *
from GalaxiesDataset import *
from rsa_loader import *
from efigi_loader import *
from pytorchtools import EarlyStopping
from myEfigi import *

import pickle

In [3]:
from PIL import Image

remove = []
x = pd.read_csv('efigi_model.csv')
#x['Galaxy_ID'] = x['pgc_full_name'].apply(lambda x: x.split(' ')[0] + x.split(' ')[-1])
for i in x['full_pgc_name']:
    if not os.path.isfile('./efigi/pics/png/{}.png'.format(i)):
        remove.append(i)
x[~x['Galaxy_ID'].isin(remove)].to_csv('usethis_rsa.csv', index=False) 

In [4]:
transf = transforms.Compose([transforms.Resize((105, 105)),
                             transforms.RandomHorizontalFlip(p=0.5),
                             transforms.RandomRotation(degrees=(0,360)),
                             transforms.RandomVerticalFlip(p=0.5),
                             transforms.ToTensor(),
                             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                  std=[0.229, 0.224, 0.225])])

transformations = transforms.Compose([transforms.Resize((105, 105)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
    

In [5]:
train_ds = GalaxiesDataset_efigi(EFIGI_DIR, EFIGI_CSV, transform=transformations)

In [6]:
cl = []
for row in range(train_ds.classes_frame.shape[0]):
    cl.append(np.argmax(train_ds.classes_frame.loc[:,train_ds.classes_frame.columns[3:]].loc[row]))

temp = pd.DataFrame(cl,columns=['cat'])
temp['cat'] = temp['cat'].astype('category')
temp['cat'] = temp['cat'].cat.codes


/home/mohamed/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


In [7]:
y = torch.from_numpy(temp.values)
train_idx, valid_idx= train_test_split(
np.arange(len(y)),
test_size=0.4,
shuffle=True,
    random_state=42,
stratify=y)

valid_idx, test_idx= train_test_split(
np.arange(len(y[valid_idx])),
test_size=0.5,
shuffle=True,
    random_state=42,
stratify=y[valid_idx])

In [8]:
size = len(train_ds)
indices = list(range(size))
split = int(np.floor(VALIDATION_SPLIT * size))
if SHUFFLE_DS:
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

test_split = int(np.floor(VALIDATION_SPLIT * len(train_indices)))
if SHUFFLE_DS:
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = train_indices[test_split:], train_indices[:test_split]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler   = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=0,
                                                 sampler=train_sampler)
val_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=0,
                                                 sampler=val_sampler)
unseen_dl = DataLoader(train_ds,batch_size=BATCH_SIZE, num_workers=0,
                                                 sampler=test_sampler)
print("Total: {} Train_dl: {} Validation_dl: {} Unseen dl: {} ".format(size, len(train_dl),                                                                    
                                                              len(val_dl),
                                                              len(unseen_dl)))

Total: 3122 Train_dl: 47 Validation_dl: 16 Unseen dl: 16 


DEVICE = 'cpu'
device = torch.device(DEVICE)

In [9]:
#Change to 'val' if only want to evaluate
train_or_val = 'train'

In [10]:
if train_or_val == 'train':
    model = torch.load('./models/gz2_resnet50')
if train_or_val == 'val':
    model = torch.load('./models/efigi')
device = torch.device(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=0.00005)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=3,threshold=0.0001,factor=0.1, verbose=True)


In [12]:
efigi = myEfigi(train_dl, val_dl, unseen_dl, model, optimizer, scheduler, criterion,device=device,BATCH_SIZE=BATCH_SIZE)
if train_or_val == 'train':
    efigi.train(n_epochs=15)

In [14]:
#For unweighted f1 score with percentage confusion matrix ...
#efigi.show_cf(f1_='notWeighted', cf_type='percentage')

#For weighted f1 score with numbers confusion matrix ...
efigi.show_cf(f1_ = 'weighted', cf_type='numbers')

Evaluation on unseen test set beginning:


RuntimeError: shape '[-1, 9]' is invalid for input of size 400